In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import date
from dateutil import parser
import duckdb

import pandas as pd
import numpy as np
import time

In [ ]:
dates = pd.date_range(date(1993,11,21),date(2025,1,1),freq='d').strftime('%#m/%#d/%Y').tolist()

print(len(dates))

In [ ]:
table_l = []

for d in dates:
    d = parser.parse(d).strftime('%#m-%#d-%Y')
    with open("crosswords/" + d + "_crossword.html", "r") as reader:
        html_source = reader.read()
    page = BeautifulSoup(html_source, "html.parser")
    
    table = page.find("table", {"id": "PuzTable"})
    rows = table.find_all("tr")
    string = ''
    for row in rows:
        cells = row.find_all("td")
        for cell in cells:
            if len(cell.text) == 0:
                string = string + 'b'
            else:
                string = string + 'w'
    table_l.append(string)
    print(d)
 
weekdays = []   
for d in dates:
     weekdays.append(parser.parse(d).strftime('%a'))
representations_df = pd.DataFrame({'date':dates,'day':weekdays,'representation':table_l})


In [ ]:
representations_df.to_csv('crossword_representations.csv', index=False)

In [ ]:
clues_df = pd.DataFrame()

for d in dates:
    d = parser.parse(d).strftime('%#m-%#d-%Y')
    with open("crosswords/" + d + "_crossword.html", "r") as reader:
        html_source = reader.read()
    page = BeautifulSoup(html_source, "html.parser")
    
    across = page.find("div", {"id":"ACluesPan"})
    try:  
        clues = across.find_all('div')[2:]
    except:
        clues = []
        pass
    
    number_l = []
    clue_l = []
    answer_l = []

    for index, clue in enumerate(clues):
        if clue.find('a') == None:
            number_l.append(clue.text)
        else:
            partition = clue.text.rpartition(':')
            clue_l.append(partition[0])
            answer_l.append(partition[2])
            
    across_df = pd.DataFrame({'number':number_l,'clue':clue_l,'answer':answer_l})
    across_df['direction'] = 'across'

    down = page.find("div", {"id":"DCluesPan"})
    try:   
        clues = down.find_all('div')[2:]
    except:
        clues = []
        pass
    
    number_l = []
    clue_l = []
    answer_l = []

    for index, clue in enumerate(clues):
        if clue.find('a') == None:
            number_l.append(clue.text)
        else:
            partition = clue.text.rpartition(':')
            clue_l.append(partition[0])
            answer_l.append(partition[2])
            
    down_df = pd.DataFrame({'number':number_l,'clue':clue_l,'answer':answer_l})
    down_df['direction'] = 'down'

    down_across_df = pd.concat([across_df, down_df])
    down_across_df['date'] = d
    down_across_df['day'] = parser.parse(d).strftime('%a')
    
    clues_df = pd.concat([clues_df, down_across_df])
    print(d)



In [ ]:
clues_df.to_csv('crossword_clues.csv', index=False)